# Defense Evasion Threat Hunting - User Activity from Stopping Windows Defensive Services
### Author: Bintang N M
Spyware and malware remain a serious problem and Microsoft developed security services, Windows Defender and Windows Firewall, to combat this threat. In the event Windows Defender or Windows Firewall is turned off, administrators should correct the issue immediately to prevent the possibility of infection or further infection and investigate to determine if caused by crash or user manipulation.

## Use Case
### Objective
Identify event with 7036 code. Event code 7036 is a stopping services event. 

### Logs Requirements
Windows System Event

### References
MITRE references: https://car.mitre.org/analytics/CAR-2016-04-003/ 

In [1]:
import evtx
import xml.etree.ElementTree as ET
import json
import re

In [2]:
evtx_file = 'logs/Defense Evasion/DE-stopDefenseService.evtx'

#### Filtering event id 7036 which indicate service's stop or start events

In [4]:
parser = evtx.PyEvtxParser(evtx_file)
parse_json = list(parser.records_json())

temp_events = []

for pj in parse_json:
    if "7036" in pj['data']:
        temp_events+=[pj]
#         print(pj['data'])

#### Filtering windows firewall and windows defender stop services
The services name can be identified on param1 element<br>
If the service is being stopped, the param2 element value would be 'stopped'

In [13]:
temp_events2 = []

for te in temp_events:
    if json.loads(te['data'])['Event']['EventData']['param1'] in ["Windows Defender", "Windows Firewall"] and json.loads(te['data'])['Event']['EventData']['param2'] == 'stopped':
        temp_events2 += [te]

In [15]:
json.loads(temp_events2[0]['data'])

{'Event': {'#attributes': {'xmlns': 'http://schemas.microsoft.com/win/2004/08/events/event'},
  'EventData': {'Binary': '570069006E0044006500660065006E0064002F0031000000',
   'param1': 'Windows Defender',
   'param2': 'stopped'},
  'System': {'Channel': 'System',
   'Computer': 'Skify-PC',
   'Correlation': None,
   'EventID': {'#attributes': {'Qualifiers': 16384}, '#text': 7036},
   'EventRecordID': 390,
   'Execution': {'#attributes': {'ProcessID': 432, 'ThreadID': 2100}},
   'Keywords': '0x8080000000000000',
   'Level': 4,
   'Opcode': 0,
   'Provider': {'#attributes': {'EventSourceName': 'Service Control Manager',
     'Guid': '{555908d1-a6d7-4695-8e1e-26931d2012f4}',
     'Name': 'Service Control Manager'}},
   'Security': None,
   'Task': 0,
   'TimeCreated': {'#attributes': {'SystemTime': '2020-04-30T17:15:09.746195Z'}},
   'Version': 0}}}